In [2]:
import os

os.environ["OPENAI_KEY"] = "sk-bQGeRZevfQI95TPb1b1a3e80011c4f368150270f687dF163"
os.environ["OPENAI_API_BASE"] = "https://api.aihubmix.com/v1"

# OpenAI与ChatOpenAI
- gpt3.5早期在langchain中叫openAI
- chatgpt4.0后针对对话模型做了更多的优化适配,所以为了和之前的api区别,叫ChatOpenAI

## OpenAI- LLM调用

In [2]:
# LLM的调用
from langchain.llms import OpenAI
import os

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE")
)
llm.invoke("你好")

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


'，我是小智，一个人工智能助手。我可以回答你的问题，帮助你解决问题，还可以和你聊天。有什么可以帮到你的吗？'

## ChatOpenAI - chat models调用

In [4]:
#调用chatmodels，以openai为例
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
import os

chat = ChatOpenAI(
    model="gpt-4",
    # temperature为0表示结果灵活度为0, 结果固定, 当灵活度比较高时,大模型会有更丰富的回答,但是可能准确度不高
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE")
)

# 可以直接调用对话
print(chat.invoke("你好"))

# 也可以传递上下文,然后提问
messages = [
    AIMessage(role="system", content="你好，我是tomie！"),
    HumanMessage(role="user", content="你好tomie，我是狗剩!"),
    AIMessage(role="system", content="认识你很高兴!"),
    HumanMessage(role="user", content="你知道我叫什么吗？")
]
print(chat.invoke(messages))

content='你好，有什么可以帮助你的吗？' response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 9, 'total_tokens': 26}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b3ce2c83-52ef-47f5-9f70-af6fc13892b0-0'
content='你刚刚告诉我，你的名字是狗剩。' response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 62, 'total_tokens': 83}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e23139ea-74b5-48c2-9300-81325df2cb45-0'


# langchain内置的LLM支持情况

![LLMs](LLMs.png)

# stream 流式调用

In [7]:
#LLM类大模型的流式输出方法

from langchain.llms import OpenAI
import os

#构造一个llm
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    max_tokens=512,
)

for chunk in llm.stream("使用李白的风格,写一首关于秋天的诗歌"):
    print(chunk, end="", flush=False)




秋风吹过黄叶飘，枫林红叶如火烧。
天高云淡秋意浓，万木凋零似梦中。

霜降寒气渐渐来，田野稻谷金色开。
收获季节喜悦多，农人欢歌笑语中。

秋雨绵绵润万物，山川湖海皆清澈。
落叶飘零如诗意，思绪随风飘远去。

秋天啊，你是多么美，给大地带来丰收喜。
让人心中充满感激，感谢你带来的一切美。

# 追踪Token的使用: 花销控制

In [8]:
#LLM的token追踪
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
import os

#构造一个llm
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    max_tokens=512,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲一个笑话")
    print(result)
    print(cb)
    # Tokens Used: 216  总token消耗
    #     Prompt Tokens: 7   提示词的token消耗
    #     Completion Tokens: 209  结果的token消耗



有一天，小明去参加一个面试，面试官问他：“你有什么特长吗？”小明回答：“我可以用手指头数到100。”面试官觉得很奇怪，就让小明演示一下。小明开始用手指头数，数到10的时候，面试官就打断他说：“好了好了，我相信你了，你可以停下来了。”小明却不停地数，数到了100，面试官很惊讶地问：“你怎么能数到100？”小明回答：“因为我数到10的时候，我发现我的裤子拉链没拉上，我一边数一边拉上拉链，所以我就顺便数到了100。”
Tokens Used: 216
	Prompt Tokens: 7
	Completion Tokens: 209
Successful Requests: 1
Total Cost (USD): $0.00042849999999999995


# 自定义输出
- 输出函数参数
- 输出json
- 输出List
- 输出日期

In [3]:
! pip install --upgrade pydantic
# ! pip install --upgrade langchain==0.1.16

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.15
    Uninstalling pydantic-1.10.15:
      Successfully uninstalled pydantic-1.10.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
doctran 0.0.14 requires openai<0.28.0,>=0.27.8, but you have openai 1.23.1 which is incompatible.
doctran 0.0.14 requires pydantic<2.0.0,>=1.10.9, but you have pydantic 2.7.0 which is incompatible.


In [7]:
#讲笑话机器人：希望每次根据指令，可以输出一个这样的笑话(小明是怎么死的？笨死的)

from langchain.llms import OpenAI
# Pydantic 是python的一种数据模型
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator

from typing import List
import os

#构造LLM
model = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
)


#定义1个Pydantic数据模型，用来描述最终的实例结构
class Joke(BaseModel):
    #setup和punchline应该是固定的字段
    setup: str = Field(description="设置笑话的问题")
    punchline: str = Field(description="回答笑话的答案")

    #验证问题是否符合要求
    @validator("setup")
    def question_check(cls, info):
        # 如果question最后一个字符不是?,就抛出异常
        if info[-1] != "？":
            raise ValueError("不符合预期的问题格式")
        return info


#将Joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    # 输出格式渲染的设置参数
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
# | 管道符: prompt函数的输出传递给model函数的入参
# 函数组合: 可以一次执行2个函数
print(prompt.format(query="请给我讲一个笑话"))

回答用户的输入.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"description": "\u8bbe\u7f6e\u7b11\u8bdd\u7684\u95ee\u9898", "title": "Setup", "type": "string"}, "punchline": {"description": "\u56de\u7b54\u7b11\u8bdd\u7684\u7b54\u6848", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}
```
请给我讲一个笑话



/var/folders/cd/6gw212g16h3dr3bbgjtd725r0000gn/T/ipykernel_23374/1125580151.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator("setup")


In [15]:
prompt_and_model = prompt | model
out_put = prompt_and_model.invoke({"query": "给我讲一个笑话"})
print("out_put:", out_put)
parser.invoke(out_put)  # 针对返回的数据结构做校验

out_put: {"setup": "为什么猫咪总是喜欢抓老鼠？", "punchline": "因为它们觉得老鼠是小弟弟！"}


Joke(setup='为什么猫咪总是喜欢抓老鼠？', punchline='因为它们觉得老鼠是小弟弟！')

In [23]:
#LLM的输出格式化成python list形式，类似['a','b','c']

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import os

#构造LLM
model = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
)

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="列出5个{subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format(subject="中国人名称")
output = model(_input)
print(output)
#格式化
print(parser.parse(output))



李明, 王芳, 张伟, 赵敏, 刘涛
['李明', '王芳', '张伟', '赵敏', '刘涛']
